<a href="https://colab.research.google.com/github/vanshuwjoshi/Kaggle-Playground-S4E11/blob/main/Optuna_Combined_CatBoost_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.0 MB/s eta 0:00:00


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv("/content/cleaned_v1.csv")
data.head()

,Work Pressure,CGPA,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Work/Study Hours,Financial Stress,Depression,Gender_Female,Gender_Male,Age_Group,City_encoded,Student,Profession_encoded,suicidal_thoughts,family_history
0,5.0,-1.0,2.0,8.0,0.0,2.0,1.0,2.0,0,1.0,0.0,3,0.192690,0,0.048567,0,0
1,4.0,-1.0,3.0,5.0,2.0,2.0,7.0,3.0,1,0.0,1.0,0,0.142206,0,0.055649,1,0
2,5.0,9.0,2.0,6.0,0.0,2.0,3.0,1.0,1,0.0,1.0,1,0.159196,1,0.585061,1,0
3,5.0,-1.0,1.0,5.0,1.0,2.0,10.0,1.0,1,0.0,1.0,0,0.131293,0,0.055649,1,1
4,1.0,-1.0,1.0,6.0,2.0,2.0,9.0,4.0,0,1.0,0.0,1,0.125739,0,0.056610,1,1


In [5]:
data.drop(["Gender_Male"], axis=1, inplace=True)
y = data["Depression"]
X = data.drop(["Depression"], axis=1)

In [6]:
import optuna
import numpy as np
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)

In [14]:
def optimize_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 42,
        'eval_metric': 'logloss',
        'task_type': 'GPU',
    }
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        model = XGBClassifier(**params, use_label_encoder=False)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=0)
        preds = model.predict(X_val)
        scores.append(accuracy_score(y_val, preds))
    return np.mean(scores)

In [15]:
xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(optimize_xgb, n_trials=30)

[I 2024-11-28 16:17:33,085] A new study created in memory with name: no-name-d02c47bc-1c7e-42fd-bf3e-a2cf5301a94e
[I 2024-11-28 16:17:52,755] Trial 0 finished with value: 0.9372423596304194 and parameters: {'n_estimators': 290, 'max_depth': 10, 'learning_rate': 0.04576817003238959, 'subsample': 0.6745815469048475, 'colsample_bytree': 0.7358250584041877, 'gamma': 3.054226637253343}. Best is trial 0 with value: 0.9372423596304194.
[I 2024-11-28 16:18:01,207] Trial 1 finished with value: 0.9338930348258707 and parameters: {'n_estimators': 108, 'max_depth': 4, 'learning_rate': 0.033217945321393216, 'subsample': 0.6922085475790862, 'colsample_bytree': 0.8585906719982775, 'gamma': 4.149912148666686}. Best is trial 0 with value: 0.9372423596304194.
[I 2024-11-28 16:18:13,985] Trial 2 finished with value: 0.9368514570007107 and parameters: {'n_estimators': 222, 'max_depth': 5, 'learning_rate': 0.25107732240003416, 'subsample': 0.7498002883422286, 'colsample_bytree': 0.933773917033357, 'gamma':

In [17]:
def optimize_cat(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 300),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_state': 42,
        'verbose': 0,
        'task_type': 'GPU',
    }
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        model = CatBoostClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                  early_stopping_rounds=10, verbose=0)
        preds = model.predict(X_val)
        scores.append(accuracy_score(y_val, preds))
    return np.mean(scores)

In [18]:
cat_study = optuna.create_study(direction='maximize')
cat_study.optimize(optimize_cat, n_trials=30)

[I 2024-11-28 16:24:36,037] A new study created in memory with name: no-name-06aa220a-a74e-432e-b0fe-8910067fbc61
[I 2024-11-28 16:24:42,324] Trial 0 finished with value: 0.9371268656716417 and parameters: {'iterations': 243, 'depth': 9, 'learning_rate': 0.2579173400410379, 'l2_leaf_reg': 7.845807267829336}. Best is trial 0 with value: 0.9371268656716417.
[I 2024-11-28 16:24:47,092] Trial 1 finished with value: 0.9376332622601279 and parameters: {'iterations': 104, 'depth': 8, 'learning_rate': 0.14839459106646932, 'l2_leaf_reg': 5.9208828472135355}. Best is trial 1 with value: 0.9376332622601279.
[I 2024-11-28 16:24:53,411] Trial 2 finished with value: 0.9382018479033404 and parameters: {'iterations': 121, 'depth': 7, 'learning_rate': 0.1104966359613722, 'l2_leaf_reg': 2.152576038851184}. Best is trial 2 with value: 0.9382018479033404.
[I 2024-11-28 16:25:00,632] Trial 3 finished with value: 0.9381218905472638 and parameters: {'iterations': 237, 'depth': 4, 'learning_rate': 0.089391814

In [19]:
xgb_top_params = [trial.params for trial in xgb_study.best_trials[:10]]
cat_top_params = [trial.params for trial in cat_study.best_trials[:10]]

In [20]:
def train_and_average(top_params, model_class, X_train, X_test, y_train):
    probas = np.zeros((X_test.shape[0], len(top_params)))
    for i, params in enumerate(top_params):
        model = model_class(**params)
        model.fit(X_train, y_train)
        probas[:, i] = model.predict_proba(X_test)[:, 1]
    return probas.mean(axis=1)

In [32]:
xgb_probas = train_and_average(xgb_top_params, XGBClassifier,
                               X_train, X_test, y_train)
cat_probas = train_and_average(cat_top_params, CatBoostClassifier,
                               X_train, X_test, y_train)

0:	learn: 0.5565806	total: 21.4ms	remaining: 4.86s
1:	learn: 0.4612903	total: 40ms	remaining: 4.51s
2:	learn: 0.3943400	total: 58.9ms	remaining: 4.42s
3:	learn: 0.3500823	total: 77.7ms	remaining: 4.35s
4:	learn: 0.3174114	total: 99.3ms	remaining: 4.43s
5:	learn: 0.2873607	total: 119ms	remaining: 4.39s
6:	learn: 0.2668488	total: 138ms	remaining: 4.34s
7:	learn: 0.2532779	total: 155ms	remaining: 4.26s
8:	learn: 0.2427911	total: 172ms	remaining: 4.19s
9:	learn: 0.2357756	total: 190ms	remaining: 4.13s
10:	learn: 0.2264858	total: 209ms	remaining: 4.12s
11:	learn: 0.2207362	total: 230ms	remaining: 4.14s
12:	learn: 0.2138163	total: 248ms	remaining: 4.11s
13:	learn: 0.2079951	total: 267ms	remaining: 4.08s
14:	learn: 0.2032481	total: 284ms	remaining: 4.04s
15:	learn: 0.1986193	total: 302ms	remaining: 4s
16:	learn: 0.1952131	total: 320ms	remaining: 3.97s
17:	learn: 0.1924352	total: 336ms	remaining: 3.92s
18:	learn: 0.1898338	total: 354ms	remaining: 3.89s
19:	learn: 0.1877830	total: 372ms	remaini

In [33]:
final_probas = (xgb_probas + cat_probas) / 2
final_predictions = (final_probas >= 0.5).astype(int)

In [34]:
accuracy = accuracy_score(y_test, final_predictions)
print("Final Accuracy Score:", accuracy)

Final Accuracy Score: 0.9382018479033405
